# Batch prediction pipline for the model trained in the previous notebook

In [ ]:
import pickle

# Load the DataFrame
with open('ratebeer.pkl', 'rb') as file:
    df = pickle.load(file)

In [ ]:
# Select a subset of the original dataset (1000 samples)
subset_dataset = df.sample(n=1000, random_state=42)  # Adjust the size as needed

In [ ]:
# Pickle the subset of data
with open("subset_data.pkl", "wb") as file:
    pickle.dump(subset_dataset, file)


In [ ]:
import hopsworks

# Log in to Hopsworks
project = hopsworks.login()

# Get the model registry instance
mr = project.get_model_registry()

model = mr.get_model("BeerRatingPredictionModel1", version=1)

model.download()

In [ ]:
import hsml

# Hopsworks documentation import

# connect with Hopsworks.
# NOTE: to connect from an external environment, see https://docs.hopsworks.ai/3.7/user_guides/projects/api_key/create_api_key 
conn = hsml.connection()

# get Hopsworks Model Serving
ms = conn.get_model_serving()

# get deployment object
deployment = ms.get_deployment("beerratingmodeldeployment1")


In [ ]:
# get Hopsworks Model Registry
mr = conn.get_model_registry(project="skris20")
  
# get model
model = mr.get_model(deployment.model_name, deployment.model_version)

# make predictions
predictions = deployment.predict({"instances": [model.input_example]})
print(predictions)


In [ ]:
# Print the DataFrame structure to verify the columns
print(subset_data.dtypes)

# Check if there are exactly 12 columns
if len(subset_data.columns) != 12:
    print(f"Error: The DataFrame should contain 12 columns, but it has {len(subset_data.columns)} columns.")
else:
    print("The DataFrame has the correct number of columns.")


In [ ]:
import pandas as pd
import numpy as np
import pickle
import hsml

# Initialize connection to Hopsworks
conn = hsml.connection()

# Get Hopsworks Model Serving
ms = conn.get_model_serving()

# Get deployment object
deployment = ms.get_deployment("beerratingmodeldeployment1")

# Load the data that has been preprocessed and pickled
with open('ratebeer.pkl', 'rb') as file:
    data = pickle.load(file)

# Check if the DataFrame has the expected columns
if 'avg_overall' not in data.columns:
    agg_data = data.groupby('beer_beerid').agg({
        'review_overall': ['mean', 'count'],
        'review_aroma': 'mean',
        'review_palate': 'mean',
        'review_taste': 'mean',
        'review_appearance': 'mean'
    }).reset_index()
    agg_data.columns = ['beer_beerid', 'avg_overall', 'review_count', 'avg_aroma', 'avg_palate', 'avg_taste', 'avg_appearance']
    data = data.merge(agg_data, on='beer_beerid')

# Ensure correct feature count and types
expected_columns = ['beer_abv', 'beer_brewerid', 'review_appearance', 'review_aroma', 
                    'review_palate', 'review_taste', 'avg_overall', 'avg_aroma', 
                    'avg_palate', 'avg_taste', 'avg_appearance', 'review_count']
data = data[expected_columns]

# Convert any datetime columns to string format
if 'review_time' in data.columns:
    data['review_time'] = data['review_time'].dt.strftime('%Y-%m-%dT%H:%M:%S')

# Convert DataFrame to list of dictionaries for prediction
input_data = data.to_dict(orient='records')

# Check each instance has exactly 12 features (Just to make sure because of earlier error)
if all(len(item) == 12 for item in input_data):
    try:
        predictions = deployment.predict({"instances": input_data})
        print("Predictions received:", predictions)
    except Exception as e:
        print("Error during prediction:", e)
else:
    print("Data formatting error: Each instance does not have exactly 12 features.")
